# Saez (2001) Replication

In [4]:
# Load libraries
using Parameters
using CSV
using DataFrames

In [3]:
# (Some imports / temporary)
# using Pkg
# Pkg.add("CSV")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed InlineStrings ── v1.2.2
   Installed WeakRefStrings ─ v1.4.2
   Installed SentinelArrays ─ v1.3.16
   Installed FilePathsBase ── v0.9.20
   Installed CSV ──────────── v0.10.7
    Updating `~/.julia/environments/v1.7/Project.toml`
  [336ed68f] + CSV v0.10.7
    Updating `~/.julia/environments/v1.7/Manifest.toml`
  [336ed68f] + CSV v0.10.7
  [48062228] + FilePathsBase v0.9.20
  [842dd82b] + InlineStrings v1.2.2
  [91c51154] + SentinelArrays v1.3.16
  [ea10d353] + WeakRefStrings v1.4.2
Precompiling project...
  ✓ SentinelArrays
  ✓ FilePathsBase
  ✓ InlineStrings
  ✓ WeakRefStrings
  ✓ CSV
  5 dependencies successfully precompiled in 9 seconds (229 already precompiled)


In [41]:
# Read in the data
simu = DataFrame(CSV.File("data/simu.csv", header = false));
# Not completely sure what the columns are
rename!(simu, [:income, :pop, :z0, :τ0]);

Data manipulation

In [43]:
# Manipulate the data
# Normalize population size to 1
nSimu = size(simu)[1];
simu.pop = simu.pop / sum(simu.pop);

# Pareto distribution above z0 = 200,000
simu_top = simu[simu.income .< 200000,:]; # Same as aux in Saez
nLower = size(simu_top)[1];
p_top = sum(simu.pop[size(simu_top)[1]+1:nSimu]);

# Compute the distribution of households
fn = simu_top.pop;
Fn = zeros(length(fn));
Fn[1] = fn[1];
for i in 2:nLower
    Fn[i] = Fn[i-1] + fn[i];
end

# Smooth out the other variables
z0 = simu_top.z0; # What is this variable?
τ0 = simu_top.τ0;
τ0 = 0.41 * ones(nLower) # Why?
τ_new = τ0
τ_old = τ0

for j in 1:1000
    for i in 2:nLower-1
        τ_new[i] = τ_old[i-1]*0.3 .+ τ_old[i]*0.4 .+ τ_old[i+1]*0.3
    end
    τ_old = τ_new
end

τ0 = τ_new


493-element Vector{Float64}:
     29.83158875
    114.3096695
    189.6199341
    257.5508118
    335.9827271
    416.074707
    508.9060974
    589.6470947
    673.6165161
    769.5877686
    860.4642944
    952.9014893
   1028.957642
      ⋮
 133821.6875
 137333.7344
 141318.1094
 145551.6094
 150170.6563
 155264.0625
 161101.6719
 167530.125
 174727.3125
 183790.375
 193995.6406
 206069.875

In [ ]:
# Set some parameters
struct params
    γ ::Int64
    k ::Int64
    e ::Float64 = 1/k
    a ::Int64
    R ::Int64
    precis ::Float64
end